In [ ]:
import plotly.graph_objects as go
import plotly.io as pio
import numpy as np
import itertools
from copy import deepcopy
from typing import Tuple, List
import pandas as pd
import sys
import os

# Set renderer to browser
pio.renderers.default = "browser"

# Ensure the src directory is in sys.path for local imports
notebook_path = os.path.abspath("")
project_root = os.path.abspath(os.path.join(notebook_path, ".."))  # parent of 'src'
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# Now import your modules
from src.local_search.rules_engine import calculate_full_score
from src.base_model.compatibility_checks import initialize_compatibility_matricies
from src.base_model.schedule import Schedule
from src.base_model.appointment import Appointment

def create_enhanced_landscape_with_better_colors(schedule, meeting_to_analyze=None, fixed_timeslot=None, normalization_scale=10000):
    """
    Enhanced landscape with better color variation by normalizing scores to larger range
    """
    
    if meeting_to_analyze is None:
        if schedule.unplanned_meetings:
            meeting_to_analyze = schedule.unplanned_meetings[0]
        else:
            planned_meetings = schedule.get_all_planned_meetings()
            if planned_meetings:
                meeting_to_analyze = planned_meetings[0]
            else:
                return None
    
    judges = schedule.get_all_judges()
    rooms = schedule.get_all_rooms()
    total_timeslots = schedule.work_days * schedule.timeslots_per_work_day
    
    if fixed_timeslot is None:
        fixed_timeslot = total_timeslots // 2
    
    day = (fixed_timeslot // schedule.timeslots_per_work_day) + 1
    timeslot_in_day = (fixed_timeslot % schedule.timeslots_per_work_day) + 1
    
    print(f"Creating enhanced landscape for Day {day}, Timeslot {timeslot_in_day}")
    print(f"Using normalization scale: 0-{normalization_scale}")
    
    # Calculate scores
    judge_indices = []
    room_indices = []
    raw_scores = []
    hover_texts = []
    
    original_schedule = deepcopy(schedule)
    
    for i, judge in enumerate(judges):
        for j, room in enumerate(rooms):
            temp_schedule = deepcopy(original_schedule)
            
            if meeting_to_analyze in temp_schedule.unplanned_meetings:
                temp_schedule.unplanned_meetings.remove(meeting_to_analyze)
            
            temp_appointment = Appointment(
                meeting=meeting_to_analyze,
                judge=judge,
                room=room,
                day=day,
                timeslot_in_day=timeslot_in_day
            )
            
            temp_schedule.add_meeting_to_schedule(temp_appointment)
            
            try:
                score = calculate_full_score(temp_schedule)[0]
            except:
                score = -1000
            
            judge_indices.append(i)
            room_indices.append(j)
            raw_scores.append(score)
            
            hover_text = (f'Judge: {judge.judge_id}<br>'
                         f'Room: {room.room_id}<br>'
                         f'Day: {day}<br>'
                         f'Timeslot: {timeslot_in_day}<br>'
                         f'Raw Score: {score:.2f}')
            hover_texts.append(hover_text)
    
    print(f"Raw score range: {min(raw_scores):.2f} to {max(raw_scores):.2f}")
    
    # NORMALIZE SCORES to 0-normalization_scale range for much better color variation
    min_score = min(raw_scores)
    max_score = max(raw_scores)
    score_range = max_score - min_score
    
    # Convert to 0-normalization_scale range
    normalized_scores = []
    for score in raw_scores:
        if score_range > 0:
            normalized = ((score - min_score) / score_range) * normalization_scale
        else:
            normalized = normalization_scale // 2  # If all scores are the same
        normalized_scores.append(normalized)
    
    print(f"Normalized score range: {min(normalized_scores):.2f} to {max(normalized_scores):.2f}")
    
    # Add MORE interpolated points for smoother layers
    import random
    random.seed(42)
    
    extra_judge_indices = []
    extra_room_indices = []
    extra_normalized_scores = []
    extra_raw_scores = []
    extra_hover_texts = []
    
    # Create a denser grid of interpolated points
    for i in range(len(judges) - 1):
        for j in range(len(rooms) - 1):
            # Get indices for this 2x2 grid
            idx_00 = i * len(rooms) + j
            idx_01 = i * len(rooms) + (j + 1) if (j + 1) < len(rooms) else idx_00
            idx_10 = (i + 1) * len(rooms) + j if (i + 1) < len(judges) else idx_00
            idx_11 = (i + 1) * len(rooms) + (j + 1) if (i + 1) < len(judges) and (j + 1) < len(rooms) else idx_00
            
            if all(idx < len(normalized_scores) for idx in [idx_00, idx_01, idx_10, idx_11]):
                # Interpolate within this cell
                for sub_i in range(3):  # More subdivisions
                    for sub_j in range(3):
                        if sub_i == 0 and sub_j == 0:
                            continue  # Skip original point
                        
                        interp_judge = i + sub_i * 0.33 + random.uniform(-0.1, 0.1)
                        interp_room = j + sub_j * 0.33 + random.uniform(-0.1, 0.1)
                        
                        # Bilinear interpolation of normalized score
                        weight_i = sub_i * 0.33
                        weight_j = sub_j * 0.33
                        
                        interp_normalized = (normalized_scores[idx_00] * (1-weight_i) * (1-weight_j) + 
                                           normalized_scores[idx_01] * (1-weight_i) * weight_j + 
                                           normalized_scores[idx_10] * weight_i * (1-weight_j) + 
                                           normalized_scores[idx_11] * weight_i * weight_j)
                        
                        # Add some controlled noise for variation (scaled to new range)
                        noise_range = normalization_scale * 0.08  # 2% of the scale
                        interp_normalized += random.uniform(-noise_range, noise_range)
                        interp_normalized = max(0, min(normalization_scale, interp_normalized))  # Clamp to range
                        
                        # Convert back to raw scale for hover
                        interp_raw = min_score + (interp_normalized / normalization_scale) * score_range
                        
                        extra_judge_indices.append(interp_judge)
                        extra_room_indices.append(interp_room)
                        extra_normalized_scores.append(interp_normalized)
                        extra_raw_scores.append(interp_raw)
                        extra_hover_texts.append(f'Interpolated<br>Score: {interp_raw:.2f}<br>Normalized: {interp_normalized:.0f}')
    
    # Combine all points
    all_judge_indices = judge_indices + extra_judge_indices
    all_room_indices = room_indices + extra_room_indices
    all_normalized_scores = normalized_scores + extra_normalized_scores
    all_raw_scores = raw_scores + extra_raw_scores
    all_hover_texts = hover_texts + extra_hover_texts
    
    # Create height based on normalized scores
    height_scores = []
    for norm_score in all_normalized_scores:
        # Use square root for more dramatic height differences
        height = (norm_score / normalization_scale) ** 0.7 * 300  # Changed from 1.5 to 0.7
        height_scores.append(height)
    
    print(f"Height range: {min(height_scores):.2f} to {max(height_scores):.2f}")
    
    # Create the landscape
    fig = go.Figure(data=go.Scatter3d(
        x=all_judge_indices,
        y=all_room_indices,
        z=height_scores,
        mode='markers',
        marker=dict(
            color=all_normalized_scores,  # Use normalized scores for color
            colorscale='RdYlBu_r',
            size=8,
            opacity=0.9,
            line=dict(width=0.2, color='rgba(0,0,0,0.2)'),
            colorbar=dict(
                title=f"Normalized Score (0-{normalization_scale})",
                thickness=25,
                len=0.8,
                tickfont=dict(size=12),
                tickmode='linear',
                tick0=0,
                dtick=normalization_scale // 10  # Show 10 tick marks across the range
            ),
            showscale=True,
            sizemode='diameter',
            cmin=0,
            cmax=normalization_scale  # Fixed color scale 0-normalization_scale
        ),
        text=all_hover_texts,
        hovertemplate='%{text}<extra></extra>'
    ))
    
    fig.update_layout(
        title=dict(
            text=f'Enhanced Landscape: Meeting {meeting_to_analyze.meeting_id} (Day {day}, Slot {timeslot_in_day})<br>Normalized Scores (0-{normalization_scale})',
            font=dict(size=16)
        ),
        scene=dict(
            xaxis_title='Judge Index',
            yaxis_title='Room Index', 
            zaxis_title='Score Height (0-300)',
            camera=dict(eye=dict(x=1.8, y=1.8, z=1.5)),
            bgcolor='rgba(15,15,15,1)',
            zaxis=dict(
                tickmode='linear',
                tick0=0,
                dtick=10,  # Z-axis steps by 10
                range=[0, 300]  # Explicit range
            )
        ),
        width=2000,
        height=1400
    )
    
    return fig

# Usage:
if __name__ == "__main__":
    from src.util.data_generator import generate_test_data_parsed
    from src.construction.heuristic.linear_assignment import generate_schedule

    parsed_data = generate_test_data_parsed(50, 5, granularity=5, min_per_work_day=390)
    initial_schedule = generate_schedule(parsed_data)
    initial_schedule.initialize_appointment_chains()
    initialize_compatibility_matricies(parsed_data)

    # Try different normalization scales:
    
    # Option 1: 10,000 scale (recommended)
    #fig1 = create_enhanced_landscape_with_better_colors(initial_schedule, fixed_timeslot=20, normalization_scale=10000)
    #fig1.show()
    
    # Option 2: 100,000 scale (even more granular)
    fig2 = create_enhanced_landscape_with_better_colors(initial_schedule, fixed_timeslot=20, normalization_scale=100000)
    fig2.show()
    
    # Option 3: 1,000 scale (less granular but still better than 100)
    # fig3 = create_enhanced_landscape_with_better_colors(initial_schedule, fixed_timeslot=20, normalization_scale=1000)
    # fig3.show()

Number of meetings: 53
Creating enhanced landscape for Day 1, Timeslot 21
Using normalization scale: 0-10000000
Raw score range: 2680268100075.00 to 2700268200076.00
Normalized score range: 0.00 to 10000000.00
Height range: 0.00 to 300.00
